In [ ]:
from secret import orkey

In [ ]:
import requests
import json

response = requests.get(
    url="https://openrouter.ai/api/v1/auth/key",
    headers={"Authorization": f"Bearer {orkey}"},
)

print(json.dumps(response.json(), indent=2))

{
  "data": {
    "label": "sk-or-v1-d2e...a95",
    "limit": null,
    "usage": 0,
    "is_provisioning_key": false,
    "limit_remaining": null,
    "is_free_tier": false,
    "rate_limit": {
      "requests": 180,
      "interval": "10s"
    }
  }
}


In [ ]:
import os, sys, re, json, asyncio, requests
from io import StringIO
from math import ceil
import xml.etree.ElementTree as ET

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.io.file_converter import xml_to_json, process_files_in_directory
from src.io.data_processor import (
    modify_json,
    save_features,
    process_data_to_structured_format,
    process_file_to_structured_format,
    process_folder_to_structured_format,
)
from src.io.dataset_process import (
    create_patients_table,
    create_ward_list_table,
    create_table_generic,
    filter_dataframe_by_patterns,
    expand_table_column,
    update_dataframe_by_patterns,
)

from src.parsers.patient_parser import (
    get_sex,
    get_age,
    get_amnez_d,
    get_amnez_life,
    get_condition,
    parse_conditions_as_key_value,
    get_structured_condition,
)
from src.parsers.hosp_parser import get_gosp_info, get_diagnosis
from src.parsers.ward_parser import (
    get_ward_table,
    get_ward_list,
    get_ward_name,
    get_research_list,
    get_research_name,
    get_research_table,
    compute_full_wards,
)
from src.parsers.final_parser import get_final_table1, get_final_table2
from src.parsers.lab_parser import get_table_1
from src.parsers.base_parser import get_full_path, SUB_PATH

from src.utils.table_utils import (
    parse_table,
    parse_table_2,
    parse_table_wtheader,
    convert_table_to_dataframe,
    safe_parse_table,
    save_table_as_dict,
    build_dataframe_from_jsons,
)
from src.utils.analysis_utils import (
    analyze_json_values,
    plot_value_distribution,
    find_files_with_value,
)
from src.utils.helpers import find_section_by_optimized_path, clean_keys

In [ ]:
import os
from openai import OpenAI
import json


os.environ["OPENAI_API_KEY"] = orkey
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
)

In [ ]:
def extract_json(content):
    match = re.search(r"\{.*\}", content, re.DOTALL)
    if match:
        json_text = match.group(0)
        return json.loads(json_text)
    else:
        raise ValueError("No JSON found.")


def generate_prompt(ecg_text, features):
    return (
        f"Извлеки значения по следующим признакам из текста ЭКГ. "
        f"Верни результат в формате JSON со всеми ключами, даже если значение отсутствует.\n\n"
        f"Если признак упомянут в тексте — заполни его как 'да/нет' или числом, если есть. "
        f"Если признака нет в тексте — поставь туда NAN.\n\n"
        f"Структура:\n{json.dumps(features, ensure_ascii=False, indent=2)}\n\n"
        f"Текст:\n{ecg_text}"
    )


async def query_func(ecg_text):
    prompt = generate_prompt(ecg_text, features)

    response = await asyncio.to_thread(
        client.chat.completions.create,
        model="google/gemini-2.0-flash-001",
        # model="google/gemini-2.0-flash-lite-001",
        messages=[
            {
                "role": "system",
                "content": "Ты в роли полезного парсера медицинских документов",
            },
            {"role": "user", "content": prompt},
        ],
        response_format=features,
        temperature=0.0,
        max_tokens=4096,
    )
    return response.choices[0].message.content


async def process_chunk(df_chunk, input_col, output_col, id_col, query_func, pbar):
    for idx, row in df_chunk.iterrows():
        try:
            result = await query_func(row[input_col])
            result = extract_json(result)

            # Add dynamic ID column as key in result
            result["observation_id"] = row[id_col]

            df_chunk.at[idx, output_col] = result
        except Exception as e:
            print(f"Ошибка при обработке строки {idx}: {e}")
            df_chunk.at[idx, output_col] = None
        finally:
            pbar.update(1)
    return df_chunk


async def process_dataframe_chunked(
    df, input_col, output_col, id_col, query_func, n_parallel=10
):
    total_rows = len(df)
    chunk_size = ceil(total_rows / n_parallel)

    tasks = []

    with tqdm(total=total_rows, desc="Обработка строк", unit="строка") as pbar:
        for start_idx in range(0, total_rows, chunk_size):
            df_chunk = df.iloc[start_idx : start_idx + chunk_size].copy()
            task = asyncio.create_task(
                process_chunk(df_chunk, input_col, output_col, id_col, query_func, pbar)
            )
            tasks.append(task)

        chunks = await asyncio.gather(*tasks)

    final_df = pd.concat(chunks).sort_index()
    return final_df


def run_processing_chunked(
    df, input_col, output_col, id_col, query_func, n_parallel=10
):
    df_copy = df.copy()
    df_copy[output_col] = None
    final_df = asyncio.run(
        process_dataframe_chunked(
            df_copy, input_col, output_col, id_col, query_func, n_parallel
        )
    )
    return final_df


def expand_and_clean_dict_column(df: pd.DataFrame, dict_column: str) -> pd.DataFrame:
    if dict_column not in df.columns:
        raise ValueError(f"Column '{dict_column}' not found in DataFrame.")

    # Expand the dictionaries into a separate DataFrame
    expanded_df = pd.json_normalize(df[dict_column])
    expanded_df.index = df.index  # align index with original df

    # Define cleaning function
    def clean_value(val):
        if isinstance(val, str):
            val = val.strip().lower()
            if val == "да":
                return True
            elif val == "нет":
                return False
            elif val == "nan":
                return np.nan
            else:
                # Try to convert to number if possible
                try:
                    if "." in val:
                        return float(val)
                    else:
                        return int(val)
                except ValueError:
                    return val  # leave as string if can't convert
        return val  # leave as is if already numeric or None

    # Apply cleaning function elementwise
    expanded_df = expanded_df.applymap(clean_value)

    # return expanded_df
    final_df = pd.concat([df.drop(columns=[dict_column]), expanded_df], axis=1)
    return final_df


def count_nans_per_column(df: pd.DataFrame) -> dict:
    """
    Counts the number of NaN values in each column.

    Args:
        df (pd.DataFrame): The DataFrame to analyze.

    Returns:
        dict: Dictionary mapping column names to their NaN count.
    """
    total_rows = len(df)
    nan_shares = (df.isna().sum() / total_rows).to_dict()
    return nan_shares


def plot_nan_counts(nan_counts: dict):
    """
    Plots a horizontal bar chart of NaN *shares* per column,
    with a taller figure.
    """
    if not nan_counts:
        print("No NaN counts to plot.")
        return

    # Sort descending
    sorted_items = sorted(nan_counts.items(), key=lambda x: x[1], reverse=True)
    cols, shares = zip(*sorted_items)

    # Make it tall: e.g. width=8, height=16
    plt.figure(figsize=(12, 16))
    plt.barh(cols, shares)
    plt.gca().invert_yaxis()  # largest bar at the top

    plt.xlabel("Share of NaN Values")
    plt.ylabel("Column Name")
    plt.title("NaN Share per Column")
    plt.grid(axis="x", linestyle="--", alpha=0.5)
    plt.tight_layout()
    plt.show()

In [ ]:
df = pd.read_csv("wl_obser_expanded_cleaned.csv")
ecg_records = df[df["observation"] == "ecg"].iloc[:500, :]


features = {
    "РИТМ_СИНУСОВЫЙ": "",
    "РИТМ_ФИБРИЛЛЯЦИЯ_ПРЕДСЕРДИЙ": "",
    "РИТМ_ЭКС": "",
    "РИТМ_ПРЕДСЕРДНЫЙ": "",
    "СИНУСОВАЯ_БРАДИКАРДИЯ": "",
    "СИНУСОВАЯ_ТАХИКАРДИЯ": "",
    "ЧСС": "",
    "ЧСС_БРАДИКАРДИЯ": "",
    "ЧСС_ТAХИКАРДИЯ": "",
    "ЭОС_НОРМАЛЬНАЯ": "",
    "ЭОС_ВЛЕВО": "",
    "ЭОС_ВПРАВО": "",
    "ЭОС_ВЕРТИКАЛЬНАЯ": "",
    "ЭОС_ГОРИЗОНТАЛЬНАЯ": "",
    "ЭОС_ПОЛУВЕРТИКАЛЬНАЯ": "",
    "БЛОКАДА_ПНПГ": "",
    "БЛОКАДА_ЛНПГ": "",
    "БЛОКАДА_ПЕРЕДНЕЙ_ВЕТВИ": "",
    "НЕПОЛНАЯ_БЛОКАДА": "",
    "ПОЛНАЯ_БЛОКАДА": "",
    "ТРИФАСЦИКУЛЯРНАЯ_БЛОКАДА": "",
    "АВ_БЛОКАДА_1_СТ": "",
    "ИМ_ПЕРЕДНЕЙ_СТЕНКИ": "",
    "ИМ_НИЖНЕЙ_СТЕНКИ": "",
    "ИМ_ПЕРЕДНЕ_ПЕРЕГОРОДОЧНОЙ_СТЕНКИ": "",
    "ИМ_ТРАНСМУРАЛЬНЫЙ": "",
    "ПЕРЕНЕСЕННЫЙ_ИМ": "",
    "ОСТРАЯ_ФАЗА_ИМ": "",
    "НАРУШЕНИЕ_РЕПОЛЯРИЗАЦИИ": "",
    "ИНВЕРСИЯ_Т": "",
    "ДЕПРЕССИЯ_ST": "",
    "ЭЛЕВАЦИЯ_ST": "",
    "ГИПЕРТРОФИЯ_ЛЖ": "",
    "ГИПЕРТРОФИЯ_ПЖ": "",
    "ГИПЕРТРОФИЯ_МЖП": "",
    "ДИЛАТАЦИЯ_ЛЖ": "",
    "НАГРУЗКА_НА_ЛП": "",
    "НАГРУЗКА_НА_ПРЕДСЕРДИЯ": "",
    "УВЕЛИЧЕНИЕ_КАМЕР_СЕРДЦА": "",
    "ДИФФУЗНЫЕ_ИЗМЕНЕНИЯ_МИОКАРДА": "",
    "СОКРАТИТЕЛЬНАЯ_ФУНКЦИЯ_СОХРАНЕНА": "",
    "СОКРАТИТЕЛЬНАЯ_ФУНКЦИЯ_СНИЖЕНА": "",
    "ДИАСТОЛИЧЕСКАЯ_ДИСФУНКЦИЯ": "",
    "ЖЕЛУДОЧКОВАЯ_ЭКСТРАСИСТОЛИЯ": "",
    "СУПРАВЕНТРИКУЛЯРНАЯ_ЭКСТРАСИСТОЛИЯ": "",
    "ЖЭС": "",
    "МЭС": "",
    "ЖИДКОСТЬ_В_ПЕРИКАРДЕ": "",
    "ИНТЕРВАЛ_QT": "",
    "ИНТЕРВАЛ_QTc": "",
    "ИНТЕРВАЛ_PQ": "",
    "ИНТЕРВАЛ_QRS": "",
    "QT_УДЛИНЕН": "",
    "СТЕНТИРОВАНИЕ": "",
    "АКШ": "",
    "ЧКВ": "",
    "ФАКТ_ОПЕРАЦИИ": "",
}

In [ ]:
import nest_asyncio

nest_asyncio.apply()

res = run_processing_chunked(
    ecg_records,
    input_col="Заключение",
    output_col="parsed_result",
    id_col="id",
    query_func=query_func,
    n_parallel=17,
)

Обработка строк: 100%|████████████████████| 500/500 [04:05<00:00,  2.04строка/s]


In [ ]:
ecg_parsed_df = expand_and_clean_dict_column(res, "parsed_result")

ecg_parsed_df.head()

/var/folders/nt/_5rbt10n3n7bbx1hrc_115700000gn/T/ipykernel_11185/1626413539.py:117: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  expanded_df = expanded_df.applymap(clean_value)


,id,id_patient,source_file,column_name,row_index,value,Эффективная доза,Протокол,Заключение,Выявленная патология,...,ИНТЕРВАЛ_QT,ИНТЕРВАЛ_QTc,ИНТЕРВАЛ_PQ,ИНТЕРВАЛ_QRS,QT_УДЛИНЕН,СТЕНТИРОВАНИЕ,АКШ,ЧКВ,ФАКТ_ОПЕРАЦИИ,observation_id
0,0,1,file_1.json,Сведения о пребывании пациента в Приемное отде...,Электрокардиография (01.01.2019 22:25),{'{urn:hl7-org:v3}table': {'{urn:hl7-org:v3}co...,NaN,""""" \n""""\n!\n\nПРОТОКОЛ ОБСЛЕДОВАНИЯ № !!!!!!!!...","ЗАКЛЮЧЕНИЕ:\n СИНУСОВЫЙ РИТМ, ЧСС 68 В МИНУТУ....",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1,1,file_1.json,Сведения о пребывании пациента в Кардиологичес...,Электрокардиография (09.01.2019 11:19),{'{urn:hl7-org:v3}table': {'{urn:hl7-org:v3}co...,NaN,""""" \n""""\n!\n\nПРОТОКОЛ ОБСЛЕДОВАНИЯ № !!!!!!!!...","ЗАКЛЮЧЕНИЕ:\n СИНУСОВЫЙ РИТМ, ЧСС 60 В МИНУТУ....",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2,1,file_1.json,Сведения о пребывании пациента в Кардиологичес...,Электрокардиография (09.01.2019 20:09),{'{urn:hl7-org:v3}table': {'{urn:hl7-org:v3}co...,NaN,""""" \n""""\n!\n\nПРОТОКОЛ ОБСЛЕДОВАНИЯ № !!!!!!!!...","ЗАКЛЮЧЕНИЕ:\n СИНУСОВЫЙ РИТМ, ЧСС 59 В МИНУТУ....",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,3,1,file_1.json,Сведения о пребывании пациента в Кардиологичес...,Электрокардиография (09.01.2019 20:22),{'{urn:hl7-org:v3}table': {'{urn:hl7-org:v3}co...,NaN,""""" \n""""\n!\n\nПРОТОКОЛ ОБСЛЕДОВАНИЯ № !!!!!!!!...","ЗАКЛЮЧЕНИЕ:\n СИНУСОВЫЙ РИТМ, ЧСС 63 В МИНУТУ....",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,4,1,file_1.json,Сведения о пребывании пациента в Кардиологичес...,Электрокардиография (09.01.2019 22:44),{'{urn:hl7-org:v3}table': {'{urn:hl7-org:v3}co...,NaN,""""" \n""""\n!\n\nПРОТОКОЛ ОБСЛЕДОВАНИЯ № !!!!!!!!...","ЗАКЛЮЧЕНИЕ:\n СИНУСОВЫЙ РИТМ, ЧСС 65 В МИНУТУ....",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4


In [ ]:
nan_counts = count_nans_per_column(ecg_parsed_df)

plot_nan_counts(nan_counts)
# saved to data

In [ ]:
rentgen_records = df[df["observation"] == "rentgen"].iloc[:500, :]

features = {
    "РЕНТГЕНОГРАММА_ГОРИЗОНТАЛЬНОМ_ПОЛОЖЕНИИ": "",
    "РЕНТГЕНОГРАММА_В_ПОЛОЖЕНИИ_СИДЯ": "",
    "РЕНТГЕНОГРАММА_В_ПОЛОЖЕНИИ_ЛЕЖА": "",
    "БЕЗ_ОТРИЦАТЕЛЬНОЙ_ДИНАМИКИ": "",
    "ОТРИЦАТЕЛЬНАЯ_ДИНАМИКА": "",
    "БЕЗ_ОЧАГОВЫХ_ТЕНЕЙ": "",
    "БЕЗ_ИНФИЛЬТРАТИВНЫХ_ТЕНЕЙ": "",
    "БЕЗ_СВЕЖИХ_ИЗМЕНЕНИЙ": "",
    "ЛЕГОЧНЫЕ_ПОЛЯ_ПРОЗРАЧНЫЕ": "",
    "ЛЕГОЧНЫЕ_ПОЛЯ_ПОНИЖЕННАЯ_ПРОЗРАЧНОСТЬ": "",
    "ЛЕГОЧНЫЕ_ПОЛЯ_СНИЖЕНИЕ_ПНЕВМАТИЗАЦИИ": "",
    "ЛЕГОЧНЫЙ_РИСУНОК_ВЫРАЖЕН_УМЕРЕННО": "",
    "ЛЕГОЧНЫЙ_РИСУНОК_ЗА_СЧЕТ_СОСУДИСТОГО_КОМПОНЕНТА": "",
    "ЛЕГОЧНЫЙ_РИСУНОК_НЕ_ИЗМЕНЕН": "",
    "ЛЕГОЧНЫЙ_РИСУНОК_УСИЛЕН": "",
    "ЛЕГОЧНЫЙ_РИСУНОК_НЕ_УСИЛЕН": "",
    "КОРНИ_ЛЕГКИХ_СТРУКТУРНЫ_НЕ_РАСШИРЕНЫ": "",
    "КОРНИ_ЛЕГКИХ_МАЛО_СТРУКТУРНЫ": "",
    "КОРНИ_ЛЕГКИХ_ЗА_ТЕНЬЮ_СРЕДОСТЕНИЯ": "",
    "КОРНИ_ЛЕГКИХ_ПРИКРЫТЫ_ТЕНЬЮ_СРЕДОСТЕНИЯ": "",
    "КОРНИ_ЛЕГКИХ_ПОЛНОКРОВНЫЕ": "",
    "КОРНИ_ЛЕГКИХ_УПЛОТНЕНЫ": "",
    "КОРНИ_ЛЕГКИХ_НЕ_ВИЗУАЛИЗИРУЮТСЯ": "",
    "СРЕДИННАЯ_ТЕНЬ_НЕ_СМЕЩЕНА": "",
    "СРЕДИННАЯ_ТЕНЬ_СМЕЩЕНА_ВПРАВО": "",
    "СРЕДИННАЯ_ТЕНЬ_СМЕЩЕНА_ВЛЕВО": "",
    "СРЕДИННАЯ_ТЕНЬ_РАСШИРЕНА": "",
    "СЕРДЦЕ_НЕ_РАСШИРЕНО": "",
    "СЕРДЦЕ_РАСШИРЕНО": "",
    "СЕРДЦЕ_ПРОЕКЦИОННО_РАСШИРЕНО": "",
    "РАЗМЕРЫ_СЕРДЦА_НЕ_ИЗМЕНЕНЫ": "",
    "АОРТА_НЕ_РАСШИРЕНА": "",
    "АОРТА_РАСШИРЕНА": "",
    "АОРТА_ЧЕТКО_НЕ_ДИФФЕРЕНЦИРУЕТСЯ": "",
    "АОРТА_ДИФФЕРЕНЦИРУЕТСЯ": "",
    "ДУГА_АОРТЫ_ДИФФЕРЕНЦИРУЕТСЯ": "",
    "ВЫБУХАНИЕ_ВТОРОЙ_ДУГИ": "",
    "КУПОЛА_ДИАФРАГМЫ_ЧЕТКИЕ_РОВНЫЕ": "",
    "КУПОЛА_ДИАФРАГМЫ_НЕ_ДИФФЕРЕНЦИРУЮТСЯ": "",
    "КУПОЛА_ДИАФРАГМЫ_ЧАСТИЧНО_НЕЧЕТКИЕ": "",
    "РЕЛАКСАЦИЯ_ПЕРЕДНЕЙ_ЧАСТИ_КУПОЛА": "",
    "НАРУЖНЫЕ_СИНУСЫ_СВОБОДНЫЕ": "",
    "НАРУЖНЫЕ_СИНУСЫ_ЗАТЕНЕНЫ": "",
    "НАРУЖНЫЕ_СИНУСЫ_ОБРЕЗАНЫ": "",
    "НАРУЖНЫЕ_СИНУСЫ_НЕ_ДИФФЕРЕНЦИРУЮТСЯ": "",
    "ПЛЕВРО_ПЕРИКАРДИАЛЬНЫЕ_СПАЙКИ": "",
    "УТОЛЩЕНИЕ_ПЛЕВРЫ": "",
    "УТОЛЩЕНА_КОСТАЛЬНАЯ_ПЛЕВРА": "",
    "ПЕРЕЛОМЫ_РЕБЕР": "",
    "ПРОВОЛОЧНЫЕ_ШВЫ": "",
    "МЕТАЛЛИЧЕСКИЕ_СКОБЫ": "",
    "СОСТОЯНИЕ_ПОСЛЕ_ОПЕРАЦИИ": "",
    "НА_ИВЛ": "",
    "ПРОЕКЦИОННЫЕ_ИСКАЖЕНИЯ": "",
    "V_AZIGUS": "",
    # признаки из заключений
    "RG_ПРИЗНАКОВ_ОЧАГОВЫХ_ИНФИЛЬТРАТИВНЫХ_ИЗМЕНЕНИЙ_ОРГАНОВ_ГРУДНОЙ_КЛЕТКИ_НЕТ": "",
    "R_ДАННЫХ_ЗА_ОРГАНИЧЕСКУЮ_ПАТОЛОГИЮ_ОРГАНОВ_ГРУДНОЙ_КЛЕТКИ_НЕ_ПОЛУЧЕНО": "",
    "RG_ПРИЗНАКИ_ЗАСТОЯ_ПО_МАЛОМУ_КРУГУ_КРОВООБРАЩЕНИЯ": "",
    "КАРДИОМЕГАЛИЯ": "",
    "R_ПРИЗНАКОВ_ОЧАГОВО_ИНФИЛЬТРАТИВНЫХ_ИЗМЕНЕНИЙ_ЛЕГКИХ_НЕ_ВЫЯВЛЕНО": "",
}

In [ ]:
import nest_asyncio

nest_asyncio.apply()

res = run_processing_chunked(
    rentgen_records,
    input_col="Протокол",
    output_col="parsed_result",
    id_col="id",
    query_func=query_func,
    n_parallel=17,
)

Обработка строк:  15%|███                  | 73/500 [00:58<05:58,  1.19строка/s]

Ошибка при обработке строки 1793: No JSON found.


Обработка строк: 100%|████████████████████| 500/500 [06:01<00:00,  1.38строка/s]


In [ ]:
rentgen_parsed_df = expand_and_clean_dict_column(res, "parsed_result")

rentgen_parsed_df

In [ ]:
nan_counts = count_nans_per_column(rentgen_parsed_df)

plot_nan_counts(nan_counts)

In [ ]:
ultra_records = df[df["observation"] == "ultrasound"].iloc[:500, :]

features = {
    "ЖИДКОСТЬ_В_ПЛЕВРАЛЬНЫХ_ПОЛОСТЯХ": "",
    "СВОБОДНАЯ_ЖИДКОСТЬ_В_ЛЕВОМ_ДИАФРАГМАЛЬНОМ_СИНУСЕ": "",
    "СВОБОДНАЯ_ЖИДКОСТЬ_В_ПРАВОМ_ДИАФРАГМАЛЬНОМ_СИНУСЕ": "",
    "ОБЪЕМ_ЖИДКОСТИ_ЛЕВЫЙ_СИНУС_МЛ": "",
    "ОБЪЕМ_ЖИДКОСТИ_ПРАВЫЙ_СИНУС_МЛ": "",
    "СЛЕДЫ_ЖИДКОСТИ_СЛЕВА": "",
    "СЛЕДЫ_ЖИДКОСТИ_СПРАВА": "",
    "ПНЕВМОТОРАКС_СЛЕВА": "",
    "ПНЕВМОТОРАКС_СПРАВА": "",
    "ОБСЛЕДОВАНИЕ_В_ГОРИЗОНТАЛЬНОМ_ПОЛОЖЕНИИ": "",
    "ОБСЛЕДОВАНИЕ_В_ПОЛОЖЕНИИ_ЛЕЖА": "",
    "ОАР_1": "",
    "ОАР_2": "",
    "ОАР_3": "",
    "ПИН_КО_2": "",
    "ПИТ_2": "",
    "РАО_1": "",
    "РАО_2": "",
    "РАО_3": "",
}

In [ ]:
import nest_asyncio

nest_asyncio.apply()

res = run_processing_chunked(
    ultra_records,
    input_col="Протокол",
    output_col="parsed_result",
    id_col="id",
    query_func=query_func,
    n_parallel=17,
)

Обработка строк: 100%|████████████████████| 500/500 [01:50<00:00,  4.51строка/s]


In [ ]:
ultra_parsed_df = expand_and_clean_dict_column(res, "parsed_result")

ultra_parsed_df

In [ ]:
nan_counts = count_nans_per_column(ultra_parsed_df)

plot_nan_counts(nan_counts)

In [ ]:
holter_records = df[df["observation"] == "holter"].iloc[:100, :]

features = {
    "НАПРАВИВШЕЕ_ЛПУ": "",
    "ВОЗРАСТ": "",
    "ДЛИТЕЛЬНОСТЬ_ОБСЛЕДОВАНИЯ": "",
    "ОБЩЕЕ_ЧИСЛО_КАРДИОЦИКЛОВ": "",
    "ЧСС_СРЕДНЯЯ_СУТОЧНАЯ": "",
    "ЧСС_СРЕДНЕ_ДНЕВНАЯ": "",
    "ЧСС_СРЕДНЕ_НОЧНАЯ": "",
    "ЧСС_МАКСИМАЛЬНАЯ": "",
    "ВРЕМЯ_МАКСИМАЛЬНОЙ_ЧСС": "",
    "ЧСС_МИНИМАЛЬНАЯ": "",
    "ВРЕМЯ_МИНИМАЛЬНОЙ_ЧСС": "",
    "ОСНОВНОЙ_РИТМ_СИНУСОВЫЙ": "",
    "ОСНОВНОЙ_РИТМ_ФИБРИЛЛЯЦИЯ_ПРЕДСЕРДИЙ": "",
    "СИНУСОВАЯ_АРИТМИЯ": "",
    "СИНУСОВАЯ_ТАХИКАРДИЯ": "",
    "СИНУСОВАЯ_БРАДИКАРДИЯ": "",
    "АВ_БЛОКАДА_1_СТЕПЕНИ": "",
    "ПОЛНАЯ_БЛОКАДА_ПРАВОЙ_НОЖКИ_ПУЧКА_ГИСА": "",
    "ПОЛНАЯ_БЛОКАДА_ЛЕВОЙ_НОЖКИ_ПУЧКА_ГИСА": "",
    "ЭПИЗОДЫ_ЭКТОПИЧЕСКОГО_ПРЕДСЕРДНОГО_РИТМА": "",
    "ЗАМЕДЛЕНИЕ_МЕЖПРЕДСЕРДНОЙ_ПРОВОДИМОСТИ": "",
    "ЗАМЕДЛЕНИЕ_ВНУТРИЖЕЛУДОЧКОВОЙ_ПРОВОДИМОСТИ": "",
    "ЖЕЛУДОЧКОВЫЕ_ЭКСТРАСИСТОЛЫ": "",
    "КОЛИЧЕСТВО_ЖЕЛУДОЧКОВЫХ_ЭКСТРАСИСТОЛ": "",
    "СУПРАВЕНТРИКУЛЯРНЫЕ_ЭКСТРАСИСТОЛЫ": "",
    "КОЛИЧЕСТВО_СУПРАВЕНТРИКУЛЯРНЫХ_ЭКСТРАСИСТОЛ": "",
    "ЭПИЗОДЫ_ПРЕДСЕРДНОЙ_ТАХИКАРДИИ": "",
    "ПАУЗЫ": "",
    "ПАРОКСИЗМАЛЬНЫЕ_НАРУШЕНИЯ_РИТМА": "",
    "ДЕПРЕССИЯ_ST": "",
    "МАКСИМАЛЬНАЯ_ДЕПРЕССИЯ_ST": "",
    "ПОДЪЕМ_ST": "",
    "МАКСИМАЛЬНЫЙ_ПОДЪЕМ_ST": "",
    "НАРУШЕНИЯ_РЕПОЛЯРИЗАЦИИ": "",
    "НАЛИЧИЕ_ПАТОЛОГИЧЕСКИХ_Q": "",
    "УДЛИНЕНИЕ_QT": "",
    "СРЕДНИЙ_QT": "",
    "СРЕДНИЙ_QTc": "",
    "МАКСИМАЛЬНЫЙ_QT": "",
    "ЦИРКАДНЫЙ_ИНДЕКС": "",
    "СНИЖЕНИЕ_ЦИРКАДНОГО_ПРОФИЛЯ_ЧСС": "",
    "РИГИДНЫЙ_ЦИРКАДНЫЙ_ИНДЕКС": "",
    "ПРЕОБЛАДАНИЕ_СИМПАТИЧЕСКОЙ_РЕГУЛЯЦИИ": "",
    "НОРМАЛЬНАЯ_РЕГУЛЯЦИЯ_СИНУСОВОГО_РИТМА": "",
    "ПРЕПАРАТЫ_БЕТА_БЛОКАТОРЫ": "",
    "ПРЕПАРАТЫ_КОРДАРОН": "",
    "ДРУГИЕ_ПРЕПАРАТЫ": "",
}

In [ ]:
import nest_asyncio

nest_asyncio.apply()

res = run_processing_chunked(
    holter_records,
    input_col="Протокол",
    output_col="parsed_result",
    id_col="id",
    query_func=query_func,
    n_parallel=17,
)

Обработка строк: 100%|████████████████████| 100/100 [00:55<00:00,  1.81строка/s]


In [ ]:
holter_parsed_df = expand_and_clean_dict_column(res, "parsed_result")

holter_parsed_df.head()

/var/folders/nt/_5rbt10n3n7bbx1hrc_115700000gn/T/ipykernel_11185/1626413539.py:117: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  expanded_df = expanded_df.applymap(clean_value)


,id,id_patient,source_file,column_name,row_index,value,Эффективная доза,Протокол,Заключение,Выявленная патология,...,МАКСИМАЛЬНЫЙ_QT,ЦИРКАДНЫЙ_ИНДЕКС,СНИЖЕНИЕ_ЦИРКАДНОГО_ПРОФИЛЯ_ЧСС,РИГИДНЫЙ_ЦИРКАДНЫЙ_ИНДЕКС,ПРЕОБЛАДАНИЕ_СИМПАТИЧЕСКОЙ_РЕГУЛЯЦИИ,НОРМАЛЬНАЯ_РЕГУЛЯЦИЯ_СИНУСОВОГО_РИТМА,ПРЕПАРАТЫ_БЕТА_БЛОКАТОРЫ,ПРЕПАРАТЫ_КОРДАРОН,ДРУГИЕ_ПРЕПАРАТЫ,observation_id
123,152,13,file_13.json,Сведения о пребывании пациента в Кардиологичес...,Холтеровское мониторирование электрокардиограм...,{'{urn:hl7-org:v3}table': {'{urn:hl7-org:v3}co...,NaN,""""" \n""""\n!\nПРОТОКОЛ ОБСЛЕДОВАНИЯ № !!!!!!!!!!...",ЗАКЛЮЧЕНИЕ\n\nЗА ВРЕМЯ ИССЛЕДОВАНИЯ ЗАРЕГИСТРИ...,NaN,...,NaN,"1,06",True,NaN,NaN,NaN,True,True,NaN,152
427,559,47,file_47.json,Сведения о пребывании пациента в Кардиологичес...,Холтеровское мониторирование электрокардиограм...,{'{urn:hl7-org:v3}table': {'{urn:hl7-org:v3}co...,NaN,"БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ХМАО-ЮГРЫ ""! \n""""\n!\nПРО...",ЗАКЛЮЧЕНИЕ\n\nЗА ВРЕМЯ ИССЛЕДОВАНИЯ ЗАРЕГИСТРИ...,NaN,...,NaN,"1,13",True,NaN,NaN,NaN,True,True,NaN,559
491,646,53,file_53.json,Сведения о пребывании пациента в Кардиологичес...,Холтеровское мониторирование электрокардиограм...,{'{urn:hl7-org:v3}table': {'{urn:hl7-org:v3}co...,NaN,""""" \n""""\n!\nПРОТОКОЛ ОБСЛЕДОВАНИЯ № !!!!!!!!!!...",ЗАКЛЮЧЕНИЕ\n\nЗА ВРЕМЯ ИССЛЕДОВАНИЯ ЗАРЕГИСТРИ...,NaN,...,NaN,"1,01",True,NaN,True,NaN,True,NaN,NaN,646
500,658,54,file_54.json,Сведения о пребывании пациента в Кардиологичес...,Холтеровское мониторирование электрокардиограм...,{'{urn:hl7-org:v3}table': {'{urn:hl7-org:v3}co...,NaN,""""" \n""""\n!\nПРОТОКОЛ ОБСЛЕДОВАНИЯ № !!!!!!!!!!...",ЗАКЛЮЧЕНИЕ\n\nЗА ВРЕМЯ ИССЛЕДОВАНИЯ ЗАРЕГИСТРИ...,NaN,...,NaN,"1,45",False,NaN,NaN,NaN,NaN,NaN,NaN,658
517,679,57,file_57.json,Сведения о пребывании пациента в Кардиологичес...,Холтеровское мониторирование электрокардиограм...,{'{urn:hl7-org:v3}table': {'{urn:hl7-org:v3}co...,NaN,""""" \n""""\n!\nПРОТОКОЛ ОБСЛЕДОВАНИЯ № !!!!!!!!!!...",ЗАКЛЮЧЕНИЕ\n\nЗА ВРЕМЯ ИССЛЕДОВАНИЯ ЗАРЕГИСТРИ...,NaN,...,NaN,"1,1",True,NaN,NaN,NaN,True,NaN,NaN,679


In [ ]:
nan_counts = count_nans_per_column(holter_parsed_df)

plot_nan_counts(nan_counts)

In [ ]:
tomography_records = df[df["observation"] == "tomography"].iloc[:100, :]

features = {
    "ФИБРОЗНЫЕ_ИЗМЕНЕНИЯ": "",
    "ОЧАГОВЫЕ_ИЗМЕНЕНИЯ": "",
    "ИНФИЛЬТРАТИВНЫЕ_ИЗМЕНЕНИЯ": "",
    "ОЧАГОВЫЕ_ТЕНИ": "",
    "КАЛЬЦИНАТЫ": "",
    "ВОЗДУШНЫЕ_ПОЛОСТИ": "",
    "БУЛЛЫ": "",
    "МАТОВОЕ_СТЕКЛО": "",
    "АТЕЛЕКТАЗ": "",
    "КОМПРЕССИОННЫЙ_АТЕЛЕКТАЗ": "",
    "ГИДРОТОРАКС": "",
    "ПНЕВМОФИБРОЗ": "",
    "ЭМФИЗЕМА": "",
    "ПНЕВМОНИЯ": "",
    "ПНЕВМОНИЯ_ПОЛИСЕГМЕНТАРНАЯ": "",
    "ЛИМФАДЕНОПАТИЯ": "",
    "ЛИМФОУЗЛЫ_УВЕЛИЧЕНЫ": "",
    "ЛИМФОУЗЛЫ_КАЛЬЦИНИРОВАНЫ": "",
    "ПЛЕВРАЛЬНЫЙ_ВЫПОТ": "",
    "ЖИДКОСТЬ_В_ПЕРИКАРДЕ": "",
    "УТОЛЩЕНИЕ_ПЕРИКАРДА": "",
    "КИСТА_ПЕРИКАРДА": "",
    "АТЕРОСКЛЕРОЗ_АОРТЫ": "",
    "АТЕРОСКЛЕРОЗ_КОРОНАРНЫХ_АРТЕРИЙ": "",
    "СТЕНТЫ_В_КОРОНАРНЫХ_АРТЕРИЯХ": "",
    "АНЕВРИЗМА_АОРТЫ": "",
    "ГРЫЖА_ПИЩЕВОДНОГО_ОТВЕРСТИЯ_ДИАФРАГМЫ": "",
    "ПРОЛАБИРОВАНИЕ_ЖЕЛУДКА": "",
    "СКОЛИОТИЧЕСКАЯ_ДЕФОРМАЦИЯ": "",
    "ДЕГЕНЕРАТИВНЫЕ_ИЗМЕНЕНИЯ_ПОЗВОНОЧНИКА": "",
    "ОСТЕОСКЛЕРОЗ": "",
    "ПНЕВМАТИЗАЦИЯ_СНИЖЕНА": "",
    "ВАСКУЛЯРИЗАЦИЯ_ИЗМЕНЕНА": "",
    "ЛИПОМАТОЗ_СРЕДОСТЕНИЯ": "",
    "КОНСОЛИДАЦИЯ_ЛЕГОЧНОЙ_ТКАНИ": "",
    "ОЧАГОВЫЕ_ОБРАЗОВАНИЯ_ПЛЕВРЫ": "",
    "ИНКАПСУЛЯЦИЯ_ЖИРОВОЙ_КЛЕТЧАТКИ": "",
    "РИСК_АТЕРОСКЛЕРОЗА_НИЗКИЙ": "",
    "РИСК_АТЕРОСКЛЕРОЗА_ВЫСОКИЙ": "",
    "CA_SCORING": "",
    "МИТРАЛЬНАЯ_НЕДОСТАТОЧНОСТЬ": "",
    "СОСТОЯНИЕ_ПОСЛЕ_СТЕНТИРОВАНИЯ": "",
    "СОСТОЯНИЕ_ПОСЛЕ_АКШ": "",
    "ЭКЗОГЕННЫЙ_АЛЬВЕОЛИТ": "",
    "ХОБЛ": "",
    "САРКОИДОЗ": "",
    "ТУБЕРКУЛЕЗНЫЕ_ИЗМЕНЕНИЯ": "",
    "ПЕТРИФИКАТЫ_В_ЛИМФОУЗЛАХ": "",
    "ИНТУБАЦИОННАЯ_ТРУБКА": "",
    "ДВУСТОРОННИЙ_ГИДРОТОРАКС": "",
    "ОТЕК_ЛЕГКИХ": "",
    "ИНТЕРСТИЦИАЛЬНЫЕ_ИЗМЕНЕНИЯ": "",
    "КАЛЬЦИНОЗ_АОРТАЛЬНОГО_КЛАПАНА": "",
    "ФИБРОТОРАКС": "",
    "БРОНХИ_УПЛОТНЕНЫ": "",
    "СЕГМЕНТАРНЫЕ_БРОНХИ_ПРОХОДИМЫ": "",
    "ПНЕВМОНИЯ_ИНТЕРСТИЦИАЛЬНАЯ": "",
    "ПНЕВМОНИЯ_АЛЬВЕОЛЯРНАЯ": "",
    "ПНЕВМОНИЯ_ПАРАСЕПТАЛЬНАЯ": "",
    "ПНЕВМОНИЯ_ЦЕНТРИЛОБУЛЯРНАЯ": "",
    "ВОЗДУШНЫЕ_КИСТЫ": "",
    "ФИБРОЗНЫЕ_ТЯЖИ": "",
    "БУЛЛЕЗНЫЕ_ИЗМЕНЕНИЯ": "",
    "СУБСЕГМЕНТАРНЫЕ_АТЕЛЕКТАЗЫ": "",
    "ПОЛИСЕГМЕНТАРНАЯ_ИНФИЛЬТРАЦИЯ": "",
    "ФИБРОАТЕЛЕКТАЗ": "",
    "КОМПРЕССИЯ_СУБСЕГМЕНТОВ": "",
    "БРОНХИАЛЬНЫЕ_СТЕНКИ_КАЛЬЦИНИРОВАНЫ": "",
    "ПНЕВМОФИБРОЗНЫЕ_ТЯЖИ": "",
    "ВУАЛЕВИДНАЯ_ИНФИЛЬТРАЦИЯ": "",
    "УТОЛЩЕНИЕ_ПЛЕВРЫ": "",
    "ГИДРОПЕРИКАРД": "",
    "ИСКРИВЛЕНИЕ_ТРАХЕИ": "",
    "СКОПЛЕНИЕ_СЛИЗИ_В_БРОНХАХ": "",
    "ЭЛЕКТРОДЫ_В_СЕРДЦЕ": "",
    "ОБЫЗВЕСТВЛЕНИЕ_СОСУДОВ": "",
    "ПНЕВМОТОРАКС": "",
    "ОБРАЗОВАНИЕ_ЛЕГКОГО": "",
    "ПЕРИКАРД_ТОНКИЙ": "",
    "СЕРДЦЕ_УВЕЛИЧЕНО": "",
    "СЕРДЦЕ_МИТРАЛЬНОЙ_КОНФИГУРАЦИИ": "",
    "ПЕРЕНЕСЕННЫЙ_ВОСПАЛИТЕЛЬНЫЙ_ПРОЦЕСС": "",
    "АТЕЛЕКТАЗ_ТОТАЛЬНЫЙ": "",
    "ОЧАГОВЫЕ_ИЗМЕНЕНИЯ_САРКОИДОЗ": "",
    "ПНЕВМОНИЯ_ВЕРХНЕДОЛЕВАЯ": "",
    "ГРЫЖА_СРЕДОСТЕНИЯ": "",
    "ДЕГЕНЕРАТИВНО_ДИСТРОФИЧЕСКИЕ_ИЗМЕНЕНИЯ": "",
    "ОБРАЗОВАНИЕ_В_СРЕДОСТЕНИИ": "",
}

In [ ]:
import nest_asyncio

nest_asyncio.apply()

res = run_processing_chunked(
    tomography_records,
    input_col="Протокол",
    output_col="parsed_result",
    id_col="id",
    query_func=query_func,
    n_parallel=17,
)

Обработка строк:  12%|██▌                  | 12/100 [00:10<00:29,  2.96строка/s]

Ошибка при обработке строки 9428: No JSON found.


Обработка строк:  17%|███▌                 | 17/100 [00:20<01:12,  1.14строка/s]

Ошибка при обработке строки 13066: No JSON found.


Обработка строк:  23%|████▊                | 23/100 [00:20<00:29,  2.64строка/s]

...

Обработка строк:  40%|████████▍            | 40/100 [00:37<01:15,  1.25s/строка]

Ошибка при обработке строки 11964: No JSON found.


Обработка строк:  63%|█████████████▏       | 63/100 [00:50<00:12,  3.00строка/s]

Ошибка при обработке строки 2701: No JSON found.


Обработка строк: 100%|████████████████████| 100/100 [01:21<00:00,  1.23строка/s]


In [ ]:
tomography_parsed_df = expand_and_clean_dict_column(res, "parsed_result")

tomography_parsed_df.head()

/var/folders/nt/_5rbt10n3n7bbx1hrc_115700000gn/T/ipykernel_11185/1626413539.py:117: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  expanded_df = expanded_df.applymap(clean_value)


,id,id_patient,source_file,column_name,row_index,value,Эффективная доза,Протокол,Заключение,Выявленная патология,...,СЕРДЦЕ_УВЕЛИЧЕНО,СЕРДЦЕ_МИТРАЛЬНОЙ_КОНФИГУРАЦИИ,ПЕРЕНЕСЕННЫЙ_ВОСПАЛИТЕЛЬНЫЙ_ПРОЦЕСС,АТЕЛЕКТАЗ_ТОТАЛЬНЫЙ,ОЧАГОВЫЕ_ИЗМЕНЕНИЯ_САРКОИДОЗ,ПНЕВМОНИЯ_ВЕРХНЕДОЛЕВАЯ,ГРЫЖА_СРЕДОСТЕНИЯ,ДЕГЕНЕРАТИВНО_ДИСТРОФИЧЕСКИЕ_ИЗМЕНЕНИЯ,ОБРАЗОВАНИЕ_В_СРЕДОСТЕНИИ,observation_id
282,359,29,file_29.json,Сведения о пребывании пациента в Кардиологичес...,Компьютерная томография груди (17.01.2019 11:00),{'{urn:hl7-org:v3}table': {'{urn:hl7-org:v3}co...,NaN,"НА ВЕРХУШКАХ, С ОБЕИХ СТОРОН, НЕБОЛЬШИЕ ФИБРОЗ...",ЗАКЛЮЧЕНИЕ: \nКТ-КАРТИНА ЛИМФАДЕНОПАТИИ СРЕДОС...,NaN,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,359.0
483,635,52,file_52.json,Сведения о пребывании пациента в Кардиологичес...,Компьютерная томография груди (29.01.2019 18:00),{'{urn:hl7-org:v3}table': {'{urn:hl7-org:v3}co...,NaN,С ОБЕИХ СТОРОН АПИКАЛЬНЫЕ ПЛЕВРЫ С ФИБРОЗНЫМИ ...,ЗАКЛЮЧЕНИЕ: \nКТ-КАРТИНА ОЧАГОВЫХ ИЗМЕНЕНИЙ В ...,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,635.0
1060,1377,109,file_109.json,Сведения о пребывании пациента в Отделение ане...,Компьютерная томография груди (12.02.2019 14:40),{'{urn:hl7-org:v3}table': {'{urn:hl7-org:v3}co...,NaN,С ПАЦИЕНТОМ (ЕГО ЗАКОННЫМ ПРЕДСТАВИТЕЛЕМ) ПРОВ...,ЗАКЛЮЧЕНИЕ: \nКТ - КАРТИНА ОБРАЗОВАНИЯ ВЕРХНЕЙ...,NaN,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1377.0
1220,1591,130,file_130.json,Сведения о пребывании пациента в Кардиологичес...,Компьютерная томография груди (20.02.2019 11:00),{'{urn:hl7-org:v3}table': {'{urn:hl7-org:v3}co...,NaN,С ПАЦИЕНТОМ (ЕГО ЗАКОННЫМ ПРЕДСТАВИТЕЛЕМ) ПРОВ...,ЗАКЛЮЧЕНИЕ: \nКТ- КАРТИНА ИНФИЛЬТРАТИВНЫХ ИЗМ...,NaN,...,False,NaN,NaN,NaN,NaN,True,NaN,NaN,False,1591.0
2701,3527,298,file_298.json,Сведения о пребывании пациента в Отделение ане...,Компьютерная томография груди (02.04.2019 18:30),{'{urn:hl7-org:v3}table': {'{urn:hl7-org:v3}co...,NaN,С ПАЦИЕНТОМ (ЕГО ЗАКОННЫМ ПРЕДСТАВИТЕЛЕМ) ПРОВ...,ЗАКЛЮЧЕНИЕ: \nМСКТ- КАРТИНА НАИБОЛЕЕ СООТВЕТСТ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
nan_counts = count_nans_per_column(tomography_parsed_df)

plot_nan_counts(nan_counts)

Обработка одного исследования стоит около `0.000731 $` и занимает около `0.7 sec`

Обработка всех исследований из текущих 8к (70к исследований) EHR будет стоить `50 $` и займет `13 часов`